# Vibrations & Modal Analysis — Hand Calculations

Executable validation formulas for modal analysis, harmonic response, and vibration absorber design.

**Reference:** `validation.md` — Vibrations & Modal Analysis

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import eig

def error_pct(ansys_val, hand_calc):
    return abs(ansys_val - hand_calc) / abs(hand_calc) * 100

def print_comparison(name, ansys_val, hand_calc, unit, threshold=5.0):
    err = error_pct(ansys_val, hand_calc)
    status = 'PASS' if err < threshold else 'FAIL'
    print(f'{name}:')
    print(f'  Hand calc:  {hand_calc:.4f} {unit}')
    print(f'  ANSYS:      {ansys_val:.4f} {unit}')
    print(f'  Error:      {err:.2f}% [{status} — threshold {threshold}%]')
    print()

---
## 1. Single-DOF Natural Frequency

In [ ]:
# --- INPUT ---
k = 10000       # Spring stiffness [N/m]
m = 5.0         # Mass [kg]
c = 0.0         # Damping coefficient [N·s/m] (0 for undamped)

ansys_freq = 0.0    # <-- Enter ANSYS natural frequency [Hz]

# --- HAND CALC ---
omega_n = np.sqrt(k / m)                   # Natural frequency [rad/s]
f_n = omega_n / (2 * np.pi)                # Natural frequency [Hz]
T_n = 1 / f_n                              # Period [s]

print(f'ω_n = √(k/m) = √({k}/{m}) = {omega_n:.4f} rad/s')
print(f'f_n = ω_n/2π = {f_n:.4f} Hz')
print(f'T_n = 1/f_n = {T_n:.4f} s')

if c > 0:
    zeta = c / (2 * np.sqrt(k * m))        # Damping ratio
    omega_d = omega_n * np.sqrt(1 - zeta**2)  # Damped natural freq
    f_d = omega_d / (2 * np.pi)
    print(f'\nζ = c/2√(km) = {zeta:.4f}')
    print(f'ω_d = ω_n√(1-ζ²) = {omega_d:.4f} rad/s')
    print(f'f_d = {f_d:.4f} Hz')
print()

if ansys_freq > 0:
    print_comparison('Natural frequency', ansys_freq, f_n, 'Hz')

---
## 2. Multi-DOF Eigenvalue Problem

Solve [K]{x} = ω²[M]{x} for natural frequencies and mode shapes.

In [ ]:
# --- INPUT: Stiffness and Mass matrices ---
# Example: 2-DOF spring-mass system
# m1--k1--m2--k2--wall
k1, k2 = 1000, 2000        # Spring stiffnesses [N/m]
m1, m2 = 1.0, 2.0          # Masses [kg]

K = np.array([[k1 + k2, -k2],
              [-k2,      k2]])

M = np.array([[m1, 0],
              [0,  m2]])

ansys_freqs = []   # <-- Enter ANSYS frequencies [Hz], e.g., [5.2, 12.8]

# --- EIGENVALUE SOLVE ---
M_inv = np.linalg.inv(M)
eigenvalues, eigenvectors = eig(M_inv @ K)

# Sort by frequency
idx = np.argsort(eigenvalues)
eigenvalues = eigenvalues[idx].real
eigenvectors = eigenvectors[:, idx].real

print('Natural frequencies:')
for i, lam in enumerate(eigenvalues):
    omega = np.sqrt(lam)
    freq = omega / (2 * np.pi)
    print(f'  Mode {i+1}: ω = {omega:.4f} rad/s, f = {freq:.4f} Hz')

print('\nMode shapes (columns = modes):')
for i in range(len(eigenvalues)):
    mode = eigenvectors[:, i] / np.max(np.abs(eigenvectors[:, i]))  # Normalize
    print(f'  Mode {i+1}: {mode}')
print()

if ansys_freqs:
    for i, af in enumerate(ansys_freqs):
        omega = np.sqrt(eigenvalues[i])
        freq = omega / (2 * np.pi)
        print_comparison(f'Mode {i+1} frequency', af, freq, 'Hz')

---
## 3. Forced Vibration (Steady-State Amplitude)

In [ ]:
# --- INPUT ---
F0 = 100        # Force amplitude [N]
k = 10000       # Stiffness [N/m]
m = 5.0         # Mass [kg]
c = 50          # Damping coefficient [N·s/m]
omega = 30      # Driving frequency [rad/s]

ansys_amplitude = 0.0   # <-- Enter ANSYS steady-state amplitude [m]

# --- HAND CALC ---
omega_n = np.sqrt(k / m)
zeta = c / (2 * np.sqrt(k * m))
r = omega / omega_n                  # Frequency ratio

X = F0 / np.sqrt((k - m * omega**2)**2 + (c * omega)**2)
X_static = F0 / k                    # Static deflection
magnification = X / X_static         # Dynamic magnification factor

print(f'ω_n = {omega_n:.4f} rad/s')
print(f'ζ = {zeta:.4f}')
print(f'r = ω/ω_n = {r:.4f}')
print(f'\nX_static = F₀/k = {X_static*1000:.4f} mm')
print(f'X_dynamic = {X*1000:.4f} mm')
print(f'Magnification factor = {magnification:.4f}')
print()

if ansys_amplitude > 0:
    print_comparison('Amplitude', ansys_amplitude*1000, X*1000, 'mm', threshold=10.0)

---
## 4. Frequency Response Plot

In [ ]:
# --- INPUT ---
F0 = 100        # Force amplitude [N]
k = 10000       # Stiffness [N/m]
m = 5.0         # Mass [kg]
zeta_values = [0.01, 0.05, 0.1, 0.2, 0.5]  # Damping ratios to plot

# --- FREQUENCY SWEEP ---
omega_n = np.sqrt(k / m)
r = np.linspace(0.01, 3, 500)   # Frequency ratio range

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for zeta in zeta_values:
    # Magnification factor
    H = 1 / np.sqrt((1 - r**2)**2 + (2 * zeta * r)**2)
    # Phase angle
    phi = np.arctan2(2 * zeta * r, 1 - r**2)

    ax1.plot(r, H, label=f'ζ = {zeta}')
    ax2.plot(r, np.degrees(phi), label=f'ζ = {zeta}')

ax1.set_xlabel('Frequency Ratio r = ω/ω_n')
ax1.set_ylabel('Magnification Factor |H(r)|')
ax1.set_title('Frequency Response — Amplitude')
ax1.set_ylim(0, 20)
ax1.axvline(x=1, color='gray', linestyle='--', alpha=0.5)
ax1.legend()
ax1.grid(True)

ax2.set_xlabel('Frequency Ratio r = ω/ω_n')
ax2.set_ylabel('Phase Angle φ (degrees)')
ax2.set_title('Frequency Response — Phase')
ax2.axhline(y=90, color='gray', linestyle='--', alpha=0.5)
ax2.legend()
ax2.grid(True)

plt.tight_layout()
plt.savefig('../results/frequency_response.png', dpi=150, bbox_inches='tight')
plt.show()
print(f'ω_n = {omega_n:.2f} rad/s = {omega_n/(2*np.pi):.2f} Hz')

---
## 5. Transmissibility

In [ ]:
# --- INPUT ---
k = 10000       # Stiffness [N/m]
m = 5.0         # Mass [kg]
zeta = 0.05     # Damping ratio
omega_drive = 100  # Driving frequency [rad/s]

ansys_transmissibility = 0.0   # <-- Enter ANSYS transmissibility [-]

# --- HAND CALC ---
omega_n = np.sqrt(k / m)
r = omega_drive / omega_n

T_num = np.sqrt(1 + (2 * zeta * r)**2)
T_den = np.sqrt((1 - r**2)**2 + (2 * zeta * r)**2)
T = T_num / T_den

print(f'r = ω/ω_n = {omega_drive}/{omega_n:.2f} = {r:.4f}')
print(f'T = {T:.6f}')
print(f'Transmitted force fraction: {T*100:.2f}%')
if r > np.sqrt(2):
    print('→ r > √2: isolation regime (T < 1)')
else:
    print('→ r < √2: amplification regime (T > 1)')
print()

if ansys_transmissibility > 0:
    print_comparison('Transmissibility', ansys_transmissibility, T, '—')

---
## 6. Vibration Absorber Design

In [ ]:
# --- INPUT: Primary system ---
m1 = 100        # Primary mass [kg]
k1 = 40000      # Primary stiffness [N/m]
omega_disturb = 20   # Disturbing frequency [rad/s]

# Design parameters
mu = 0.1        # Mass ratio m2/m1 (typically 0.05–0.25)

# --- ABSORBER DESIGN ---
omega_1 = np.sqrt(k1 / m1)
print(f'Primary natural freq: ω₁ = {omega_1:.2f} rad/s = {omega_1/(2*np.pi):.2f} Hz')
print(f'Disturbing freq: ω = {omega_disturb:.2f} rad/s = {omega_disturb/(2*np.pi):.2f} Hz')
print()

# Absorber tuned to disturbing frequency
m2 = mu * m1                           # Absorber mass
k2 = m2 * omega_disturb**2             # Absorber stiffness (tuned)

print(f'--- Absorber Parameters ---')
print(f'Mass ratio μ = {mu}')
print(f'Absorber mass: m₂ = μ·m₁ = {m2:.1f} kg')
print(f'Absorber stiffness: k₂ = m₂·ω² = {k2:.0f} N/m')
print()

# New natural frequencies of combined 2-DOF system
K = np.array([[k1 + k2, -k2],
              [-k2,      k2]])
M = np.array([[m1, 0],
              [0,  m2]])
M_inv = np.linalg.inv(M)
eigenvalues = np.sort(np.linalg.eigvals(M_inv @ K).real)

print('Combined system natural frequencies:')
for i, lam in enumerate(eigenvalues):
    omega = np.sqrt(lam)
    print(f'  ω_{i+1} = {omega:.2f} rad/s = {omega/(2*np.pi):.2f} Hz')

print(f'\n→ At ω = {omega_disturb} rad/s, primary mass amplitude = 0 (perfectly tuned)')
print(f'→ Avoid operating between {np.sqrt(eigenvalues[0]):.1f} and {np.sqrt(eigenvalues[1]):.1f} rad/s')

---
## 7. Beam Natural Frequencies (Analytical)

Compare to ANSYS Modal analysis for beam structures.

In [ ]:
# --- INPUT ---
E = 200e9       # Young's modulus [Pa]
rho = 7850      # Density [kg/m³]
b = 0.05        # Beam width [m]
h = 0.01        # Beam height [m]
L = 1.0         # Beam length [m]
n_modes = 5     # Number of modes to compute

# Boundary condition type
bc = 'cantilever'  # 'cantilever', 'simply_supported', 'fixed_fixed'

ansys_beam_freqs = []   # <-- Enter ANSYS frequencies [Hz]

# --- HAND CALC ---
I = b * h**3 / 12
A = b * h
m_per_L = rho * A       # Mass per unit length [kg/m]

# Beta*L values for different BCs (from vibration tables)
beta_L = {
    'cantilever':        [1.8751, 4.6941, 7.8548, 10.9955, 14.1372],
    'simply_supported':  [np.pi*n for n in range(1, n_modes+1)],
    'fixed_fixed':       [4.7300, 7.8532, 10.9956, 14.1372, 17.2788],
}

print(f'BC type: {bc}')
print(f'EI = {E*I:.4f} N·m²')
print(f'ρA = {m_per_L:.4f} kg/m')
print()

betas = beta_L[bc][:n_modes]
print('Natural frequencies:')
for i, bl in enumerate(betas):
    omega = bl**2 * np.sqrt(E * I / (m_per_L * L**4))
    freq = omega / (2 * np.pi)
    print(f'  Mode {i+1}: βL = {bl:.4f}, ω = {omega:.2f} rad/s, f = {freq:.2f} Hz')

    if i < len(ansys_beam_freqs):
        print_comparison(f'  Mode {i+1}', ansys_beam_freqs[i], freq, 'Hz')